In [2]:
# all this has to do is split a list of ids into three groups
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def divide_by_three(list_of_ids):
    destinations = []
    while len(list_of_ids) > 2 or len(list_of_pairs) % 3 == 0:
        #grab a random id from big_genre and remove
        for m, l in enumerate(destinations):

            candidates = list(list_of_pairs.keys())
            shuffle(candidates)
            key = candidates[0]
            #print(key)
            candidate = list_of_pairs[key]
            #remove from dictionary
            del list_of_pairs[key]
            #make sure the author is not in the dict
            if candidate not in l:
                l[key] = candidate

            else:
                next_one = destinations[m+1]
                #next list same id if author in dict
                if candidate not in next_one:
                    next_one[key] = candidate
                else:
                    next_one = destinations[m+2]
                    if candidate not in next_one:
                        next_one[key] = candidate
    destinations_trimmed =[]
    for i,j in enumerate(destinations):
        aTuple = list(j.items())
        shuffle(aTuple)
        destinations_trimmed.append(dict(aTuple[:83]))
    return destinations_trimmed

def divide_list(mylist, number_of_splits):
    def chunks(l, n):
        """Yield successive n-sized chunks from l."""
        for i in range(0, len(l), n):
            yield l[i:i + n]

    l = (len(mylist) % number_of_splits)
    a = len(mylist) - l
    b = int(a/number_of_splits)
    newlists = list(chunks(mylist, b))
    newlists = [i for i in newlists if len(i) == b]
    return newlists

#this will be moved to application folder
def partition_test_train(genre, df, number_of_partitions):
    # supply a genre
    df_genre = df.loc[df['big_genres'] == genre]
    df_non_genre = df.loc[df['big_genres'] != genre ]
    df_non_genre = df_non_genre.loc[df_non_genre['big_genres'] != "multi" ]
    # create a dataset of random ids 50% in genre, 50% out of genre
    genre_ids = list(df_genre['ids'])
    # split into three separate sets of ids
    genre_split = divide_list(genre_ids, number_of_partitions)

    non_genre_ids = list(df_non_genre['ids'])
    #shuffle non_genre before splitting
    shuffle(non_genre_ids)
    non_genre_ids = non_genre_ids[0:len(genre_ids)]
    # split into three separate sets of ids
    non_genre_split = divide_list(non_genre_ids, number_of_partitions)

    rejoined = []
    #rejoin genre and nongenre

    for i, j in enumerate(non_genre_split):
        joined = j + genre_split[i]
        rejoined.append(joined)
    #sort by id and convert rejoined back to full df
    for r in rejoined:
        r.sort()
    rejoined_dfs = []
    for i in rejoined:
        new_df = df.loc[df['ids'].isin(i)]
        rejoined_dfs.append(new_df)
    return rejoined_dfs, rejoined

In [3]:
from random import shuffle
from application.selective_features import dictionaries_of_features, make_genres_big_and_lavin
from application.pickles import pickledData
import pandas as pd
from scipy.stats.stats import spearmanr
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np

pData = pickledData()

Loaded pickle data successfully.


In [4]:
# import feature dictionaries
feature_dicts = pData.feature_dicts
#define a dictionary with ids, genres, dates, authors, etc. Pass to each function
metadata = {"ids": pData._ids, "dates": pData.dates, "genres":pData.genres, "authors":pData.authors,
                "titles":pData.titles,"big_genres":pData.big_genres, "lavin_genres":pData.lavin_genres}
#convert all metadata to dataframe
data = [metadata[i] for i in metadata.keys()]
df = pd.DataFrame(data)
df = df.transpose()
df.index.name = 'position'
df.columns=metadata.keys()

partitioned_dfs, partitioned_ids = partition_test_train("gothic", df, 2)

#convert lists of ids into dictionaries of features in order of ids
partitioned_dicts = []

#loop enumerate(pData.ids)
for id_list in partitioned_ids:
    processing_list =[]
    for index, _id in enumerate(pData._ids):
        if _id in id_list:
            processing_list.append(feature_dicts[index])
    partitioned_dicts.append(processing_list)

In [5]:
train_genres = list(partitioned_dfs[0]['big_genres']) 
train_genres = ["gothic" if g == "gothic" else "not gothic" for g in train_genres] 
target_genres = list(partitioned_dfs[1]['big_genres']) 
target_genres = ["gothic" if g == "gothic" else "not gothic" for g in target_genres] 

In [6]:
text_clf = Pipeline([('vect', DictVectorizer()), ('tfidf', TfidfTransformer()),('clf', LogisticRegression()),])
text_clf = text_clf.fit(partitioned_dicts[0], train_genres)

In [7]:
predicted = text_clf.predict(partitioned_dicts[1])
mean_accuracy = np.mean(predicted == target_genres)

In [8]:
proba = text_clf.predict_proba(partitioned_dicts[1])

In [9]:
predicted

array(['gothic', 'gothic', 'not gothic', 'gothic', 'gothic', 'gothic',
       'gothic', 'gothic', 'gothic', 'gothic', 'gothic', 'gothic',
       'not gothic', 'gothic', 'gothic', 'gothic', 'gothic', 'not gothic',
       'gothic', 'gothic', 'not gothic', 'not gothic', 'gothic', 'gothic',
       'not gothic', 'gothic', 'gothic', 'not gothic', 'gothic',
       'not gothic', 'gothic', 'gothic', 'gothic', 'gothic', 'gothic',
       'not gothic', 'gothic', 'gothic', 'gothic', 'not gothic', 'gothic',
       'not gothic', 'not gothic', 'not gothic', 'gothic', 'not gothic',
       'not gothic', 'gothic', 'not gothic', 'not gothic', 'not gothic',
       'gothic', 'gothic', 'gothic', 'not gothic', 'not gothic', 'gothic',
       'gothic', 'not gothic', 'gothic', 'not gothic', 'not gothic',
       'not gothic', 'gothic', 'not gothic', 'not gothic', 'gothic',
       'gothic', 'not gothic', 'not gothic', 'gothic', 'not gothic',
       'not gothic', 'gothic', 'gothic', 'not gothic', 'gothic',
       '

In [202]:
text_clf.classes_

array(['gothic', 'not gothic'], 
      dtype='<U10')

In [203]:
mean_accuracy

0.6428571428571429

In [11]:
result = partitioned_dfs[1]
p = [i[0] for i in proba]
          
result["predicted"] = predicted
result["probability"] = p

from sklearn.metrics import recall_score

t = [0 if s == "gothic" else 1 for s in target_genres]
pr = [0 if s == "gothic" else 1 for s in predicted]
pr_false = [1 if s == "gothic" else 0 for s in predicted]

result["target_binary"] = t
result["predicted_binary"] = pr
result["target"] = target_genres
correct = []
for y,z in enumerate(predicted): 
    if z == target_genres[y]:
        correct.append(1)
    else:
        correct.append(0)
result["correct"] = correct  

result["predicted_binary_false"] = pr_false

result = result.sort_values("probability", ascending="False")
r_top = np.array_split(result, 2)[1]
r_top

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

,genres,titles,big_genres,authors,ids,lavin_genres,dates,predicted,probability,target_binary,predicted_binary,target,correct,predicted_binary_false
position,,,,,,,,,,,,,,
403,teamred | pbgothic | stew | lavin_ghost,Wandering+ghosts,gothic,Crawford%2C+F.+Marion,404,lavin_ghost,1911,not gothic,0.396935,0,1,gothic,0,0
460,teamred | det100,The+twenty-six+clues,crime,Ostrander%2C+Isabel%2C,461,no_lavin_tag,1919,not gothic,0.415619,1,1,not gothic,1,0
444,teamblack | random,The+worn+doorstep,non_genre,Sherwood%2C+Margaret+Polloc,445,no_lavin_tag,1916,not gothic,0.422433,1,1,not gothic,1,0
390,teamred | pbgothic | lavin_ghost,Tales+of+men+and+ghosts,gothic,Wharton%2C+Edith%2C,391,lavin_ghost,1910,not gothic,0.425690,0,1,gothic,0,0
392,teamblack | random,Westover+of+Wanalah%3B,non_genre,Eggleston%2C+George+Cary%2C,393,no_lavin_tag,1910,not gothic,0.432476,1,1,not gothic,1,0
461,teamred | pbgothic | lavin_ghost,A+thin+ghost+and+others,gothic,James%2C+M.+R.,462,lavin_ghost,1919,not gothic,0.437343,0,1,gothic,0,0
313,teamblack | random,When+London+burned,non_genre,Henty%2C+G.+A.,314,no_lavin_tag,1894,not gothic,0.440607,1,1,not gothic,1,0
419,teamred | locghost | lavin_ghost,The+Meadow-Brook+girls+un,gothic,Aldridge%2C+Janet.,420,lavin_ghost,1913,not gothic,0.445144,0,1,gothic,0,0
40,teamred | lockandkey | stew,The+Three+Sharpers,crime,Anonymous8,41,no_lavin_tag,1800,not gothic,0.449006,1,1,not gothic,1,0


In [12]:
result.groupby(['target'])['predicted_binary_false'].sum()

target
gothic        32
not gothic    17
Name: predicted_binary_false, dtype: int64

In [13]:
result.groupby(['target'])['predicted_binary'].sum()

target
gothic        38
not gothic    53
Name: predicted_binary, dtype: int64

In [14]:
recall_score(t, pr)
#how often would a category appear accurate as a matter of random chance?
#every possible combination in piles, distribution accuracy

0.75714285714285712

In [15]:
#are true positives for gothic date influenced?
#make confusion matrices
len(text_clf.named_steps['clf'].coef_[0])

9000

In [236]:
type(text_clf.named_steps['vect'].vocabulary_)

dict

In [19]:
a = text_clf.named_steps['vect'].vocabulary_.keys()

In [21]:
list(a)[:10]

['shapeless',
 'hallway',
 'staying',
 'van',
 'starving',
 'muttered',
 'wonders',
 'play',
 'assisting',
 'eccentric']